## **PSC Users**

### 1️⃣ **Step 1 Setting Up Your Environment on Bridges2**

❗️⚠️ For this homework, we are **providing shared Datasets and a shared Conda environment** for the entire class.

❗️⚠️ So for PSC users, **do not download the data yourself** and **do not need to manually install the packages**!

Follow these steps to set up the environment and start a Jupyter notebook on Bridges2:

To run your notebook more efficiently on PSC, we need to use a **Jupyter Server** hosted on a compute node.

You can use your prefered way of connecting to the Jupyter Server. 


**The recommended way of connecting is:**

#### **Connect in VSCode**
SSH into Bridges2 and navigate to your **Jet directory** (`Jet/home/<your_username>`). Upload your notebook there, and then connect to the Jupyter Server from that directory.

#### **1. SSH into Bridges2**
1）Open VS Code and click on the `Extensions` icon in the left sidebar. Make sure the "**Remote - SSH**" extension is installed.

2）Open the command palette (**Shift+Command+P** on Mac, **Ctrl+Shift+P** on Windows). A search box will appear at the top center. Choose `"Remote-SSH: Add New SSH Host"`, then enter:

```bash
ssh <your_username>@bridges2.psc.edu #change <your_username> to your username
```

Next, choose `"/Users/<your_username>/.ssh/config"` as the config file. A dialog will appear in the bottom right saying "Host Added". Click `"Connect"`, and then enter your password.

(Note: After adding the host once, you can later use `"Remote-SSH: Connect to Host"` and select "bridges2.psc.edu" from the list.)

3）Once connected, click `"Explorer"` in the left sidebar > "Open Folder", and navigate to your home directory under the project grant:
```bash
/jet/home/<your_username>  #change <your_username> to your username
```

4）You can now drag your notebook files directly into the right-hand pane (your remote home directory), or upload them using `scp` into your folder.

> ❗️⚠️ The following steps should be executed in the **VSCode integrated terminal**.

### **2. Navigate to Your Directory**
Make sure to use this `/jet/home/<your_username>` as your working directory, since all subsequent operations (up to submission) are based on this path.
```bash
cd /jet/home/<your_username>  #change <your_username> to your username
```

#### **3. Request a Compute Node**
```bash
interact -p GPU-shared --gres=gpu:v100-32:1 -t 8:00:00 -A cis250019p
```

#### **4. Load the Anaconda Module**
```bash
module load anaconda3
```

#### **5. Activate the provided HW1 Environment**
```bash
conda deactivate # First, deactivate any existing Conda environment
conda activate /ocean/projects/cis250019p/mzhang23/TA/HW1P2/envs/hw1p2_env && export PYTHONNOUSERSITE=1
```

#### **6. Start Jupyter Notebook**
Launch Jupyter Notebook:
```bash
jupyter notebook --no-browser --ip=0.0.0.0
```

Go to **Kernel** → **Select Another Kernel** → **Existing Jupyter Server**
   Enter the URL of the Jupyter Server:```http://{hostname}:{port}/tree?token={token}```
   
   *(Usually, this URL appears in the terminal output after you run `jupyter notebook --no-browser --ip=0.0.0.0`, in a line like:  “Jupyter Server is running at: http://...”)*

   - eg: `http://v011.ib.bridges2.psc.edu:8888/tree?token=e4b302434e68990f28bc2b4ae8d216eb87eecb7090526249`

> **Note**: Replace `{hostname}`, `{port}` and `{token}` with your actual values from the Jupyter output.

#### **7. Import dependencies**

After launching the Jupyter notebook, you can run the cells directly inside the notebook — no need to use the terminal for the remaining steps.

First, import the dependencies.  
(*If you followed the previous steps and correctly activated our shared `hw1p2` environment, you do **not** need to install anything manually.*)

In [ ]:
#Make sure you are in your directory
!pwd #should be /jet/home/<your_username>, if not, uncomment the following line and replace with your actual username:
# %cd <your_username>  #TODO: replace the "<your_username>" to yours

### 2️⃣ **Step 2: Set up Kaggle API Authentication**

In [ ]:
# TODO: Use the same Kaggle code from HW1P2
!mkdir /jet/home/<your_username>/.kaggle #TODO: replace the "<your_username>" to yours

with open("/jet/home/<your_username>/.kaggle/kaggle.json", "w+") as f: #TODO: replace the "<your_username>" to yours
    f.write('{"username":"<YOUR USERNAME>","key":"<YOUR KEY>"}')
    # TODO: Put your kaggle username & key here

!chmod 600 /jet/home/<your_username>/.kaggle/kaggle.json #TODO: replace the "<your_username>" to yours

### 3️⃣ **Step 3: Get Data**

❗️⚠️ The data used in this assignment is **already stored in a shared, read-only folder, so you do not need to manually download anything**.

Instead, just make sure to replace the dataset path in your notebook code with the correct path from the shared directory.

You can run the following block to explore the shared directory structure:

In [ ]:
import os
data_path = "/ocean/projects/cis250019p/mzhang23/TA/HW1P2/hw1p2_data/11785-f25-hw1p2" #Shared data path, do not need to change the username to yours
print("Files in shared hw1p2 dataset:", os.listdir(data_path))

In [ ]:
!apt-get install tree
!tree -L 2 /ocean/projects/cis250019p/mzhang23/TA/HW1P2/hw1p2_data

In [ ]:
# !pip install wandb --quiet

In [ ]:
# !pip install torchaudio --quiet

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import sklearn
import gc
import zipfile
import bisect
import pandas as pd
from tqdm.auto import tqdm
import os
import datetime
import wandb
import yaml
import torchaudio.transforms as tat
import torchaudio

if torch.backends.mps.is_available():
    device = torch.device("mps")  
elif torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

print("Device:", device) 

In [ ]:
### PHONEME LIST
PHONEMES = [
            '[SIL]',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '[SOS]', '[EOS]']

In [ ]:
config = {
    'Name': 'Jinyao Zhou', # Write your name here
    'subset': 1, # Subset of train/val dataset to use (1.0 == 100% of data)
    'context': 30,
    'archetype': 'pyramid', # Default Values: pyramid, diamond, inverse-pyramid,cylinder
    'activations': 'GeLU',
    'learning_rate': 0.001,
    'dropout': 0.15,
    'optimizers': 'AdamW',
    'scheduler': 'ReduceLROnPlateau', #ReduceLROnPlateau,CosineAnnealingLR
    'T_max': 10,
    'eta_min': 0.0002,
    'epochs': 30,
    'batch_size': 1024,
    'weight_decay': 0.001,
    'weight_initialization': "kaiming_normal", # e.g kaiming_normal, kaiming_uniform, uniform, xavier_normal or xavier_uniform
    'augmentations': 'Both', # Options: ["FreqMask", "TimeMask", "Both", null]
    'freq_mask_param': 4,
    'time_mask_param': 8,
    'step_size': 5,
    'gamma': 0.99
    }

run_name = "<CHANGE TO YOUR WANDB RUN NAME>"

In [ ]:
class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, root, phonemes, config, context=0, partition= "train-clean-100"): # Feel free to add more arguments

        self.context    = context
        self.phonemes   = phonemes
        self.subset = config['subset']

        # TODO: Initialize augmentations. Read the Pytorch torchaudio documentations on timemasking and frequencymasking
        self.freq_masking = torchaudio.transforms.FrequencyMasking(freq_mask_param=config['freq_mask_param'])
        self.time_masking = torchaudio.transforms.TimeMasking(time_mask_param=config['time_mask_param'])

        # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data using root
        self.mfcc_dir       = root + '/' + partition + '/mfcc'
        # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data using root
        self.transcript_dir = root +'/' + partition + '/transcript'

        # TODO: List files in sefl.mfcc_dir using os.listdir in SORTED order
        mfcc_names          = os.listdir(self.mfcc_dir)
        mfcc_names.sort()
        # TODO: List files in self.transcript_dir using os.listdir in SORTED order
        transcript_names    = os.listdir(self.transcript_dir)
        transcript_names.sort()

        # Compute size of data subset
        subset_size = int(self.subset * len(mfcc_names))

        # Select subset of data to use
        mfcc_names = mfcc_names[:subset_size]
        transcript_names = transcript_names[:subset_size]

        # Making sure that we have the same no. of mfcc and transcripts
        assert len(mfcc_names) == len(transcript_names)

        self.mfccs, self.transcripts = [], []

        # TODO: Iterate through mfccs and transcripts
        for i in tqdm(range(len(mfcc_names))):

            mfcc             = np.load(os.path.join(self.mfcc_dir, mfcc_names[i]))
            mfccs_normalized = (mfcc - np.mean(mfcc, axis=0, keepdims=True)) / np.std(mfcc, axis=0, keepdims=True)
            mfccs_normalized = torch.tensor(mfccs_normalized, dtype=torch.float32)

            transcript  = np.load(os.path.join(self.transcript_dir, transcript_names[i]), allow_pickle=True).tolist()
            transcript  = transcript[1:-1]
            transcript_indices = [self.phonemes.index(phoneme) for phoneme in transcript] # map string to number
            transcript_indices = torch.tensor(transcript_indices, dtype=torch.int64)

            self.mfccs.append(mfccs_normalized)
            self.transcripts.append(transcript_indices)

        # concatenate along time T
        self.mfccs          = torch.cat(self.mfccs, dim=0)
        self.transcripts    = torch.cat(self.transcripts, dim=0)
        self.length = len(self.mfccs) # len = time duration

        self.mfccs = torch.nn.functional.pad(self.mfccs, (0, 0, context, context)) # the trick is you don't need to update the length of mfcc


    def __len__(self):
        return self.length

    def collate_fn(self, batch):
        x, y = zip(*batch) # unpair x and y one by one
        x = torch.stack(x, dim=0)

        # agumentation
        if np.random.rand() < 0.1: #TODO: changable
            x = x.transpose(1, 2)  # Shape: (batch_size, freq, time)
            x = self.freq_masking(x)
            x = self.time_masking(x)
            x = x.transpose(1, 2)  # Shape back to: (batch_size, time, freq)
            
        return x, torch.tensor(y)

    def __getitem__(self, ind):     
        frames = self.mfccs[ind:ind+2*self.context+1]
        phonemes = self.transcripts[ind]
        return frames, phonemes

In [ ]:

class AudioTestDataset(torch.utils.data.Dataset):

    # Create a test dataset class similar to the previous class but you dont have transcripts for this
    # Imp: Read the mfccs in sorted order, do NOT shuffle the data here or in your dataloader.
    # IMPORTANT: Load complete test data to use, DO NOT select subset of test data, else you will get errors when submitting on Kaggle.

    def __init__(self, root, phonemes, config, context=0, partition= "test-clean"): # Feel free to add more arguments

        self.context    = context
        self.phonemes   = phonemes

        self.freq_masking = torchaudio.transforms.FrequencyMasking(freq_mask_param=config['freq_mask_param'])
        self.time_masking = torchaudio.transforms.TimeMasking(time_mask_param=config['time_mask_param'])

        self.mfcc_dir       = root + '/' + partition + '/mfcc'

        # TODO: List files in sefl.mfcc_dir using os.listdir in SORTED order
        mfcc_names          = os.listdir(self.mfcc_dir)
        mfcc_names.sort()

        self.mfccs = []

        for i in tqdm(range(len(mfcc_names))):
            mfcc             = np.load(os.path.join(self.mfcc_dir, mfcc_names[i]))
            mfccs_normalized = (mfcc - np.mean(mfcc, axis=0, keepdims=True)) / np.std(mfcc, axis=0, keepdims=True)
            mfccs_normalized = torch.tensor(mfccs_normalized, dtype=torch.float32)
            self.mfccs.append(mfccs_normalized)

        self.mfccs = torch.cat(self.mfccs, dim=0)
        self.length = len(self.mfccs)
        self.mfccs = torch.nn.functional.pad(self.mfccs, (0, 0, context, context))

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        frames = self.mfccs[ind:ind+2*self.context+1]
        return frames

In [ ]:
# ROOT = "/kaggle/input/11785-hw-1-p-2-fall-2025-main/11785-f25-hw1p2/" # Define the root directory of the dataset here
ROOT = data_path
train_data = AudioDataset(ROOT, PHONEMES, config, context=config['context'])
val_data = AudioDataset(ROOT, PHONEMES, config, context=config['context'], partition="dev-clean")
test_data = AudioTestDataset(ROOT, PHONEMES, config, context=config['context'])

In [ ]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
# We shuffle train dataloader but not val & test dataloader. Why?

train_loader = torch.utils.data.DataLoader(
    dataset     = train_data,
    num_workers = 4, #change for differnet device
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = True,
    collate_fn = train_data.collate_fn
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data,
    num_workers = 0,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test_data,
    num_workers = 0,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)


print("Batch size     : ", config['batch_size'])
print("Context        : ", config['context'])
print("Input size     : ", (2*config['context']+1)*28)
print("Output symbols : ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

In [ ]:
# import matplotlib.pyplot as plt

# # Testing code to check if your data loaders are working
# for i, data in enumerate(train_loader):
#     frames, phoneme = data
#     print(frames.shape, phoneme.shape)

#     # Visualize sample mfcc to inspect and verify everything is correctly done, especially augmentations
#     plt.figure(figsize=(10, 6))
#     plt.imshow(frames[0].numpy().T, aspect='auto', origin='lower', cmap='viridis')
#     plt.xlabel('Time')
#     plt.ylabel('Features')
#     plt.title('Feature Representation')
#     plt.show()
    
#     break

In [ ]:
# Testing code to check if your validation data loaders are working
all = []
for i, data in enumerate(val_loader):
    frames, phoneme = data
    all.append(phoneme)
    print(frames.shape, phoneme.shape)
    break

In [ ]:
class Network(nn.Module):
    def __init__(self, input_size, output_size, config):
        super(Network, self).__init__()

        self.weight_init = config["weight_initialization"]
        self.dropout = config["dropout"]

        self.model = nn.Sequential(
            nn.Linear(input_size, 2048),      
            nn.BatchNorm1d(2048),
            # nn.LeakyReLU(),
            nn.GELU(),
            nn.Dropout(p=self.dropout*1.25),

            nn.Linear(2048, 2048),            
            nn.BatchNorm1d(2048),
            # nn.LeakyReLU(),
            nn.GELU(),
            nn.Dropout(p=self.dropout*1.25),

            nn.Linear(2048, 1536),            
            nn.BatchNorm1d(1536),
            # nn.LeakyReLU(),
            nn.GELU(),
            nn.Dropout(p=self.dropout),

            nn.Linear(1536, 1024),            
            nn.BatchNorm1d(1024),
            # nn.LeakyReLU(),
            nn.GELU(),
            nn.Dropout(p=self.dropout),

            nn.Linear(1024, 1024),            
            nn.BatchNorm1d(1024),
            # nn.LeakyReLU(),
            nn.GELU(),
            nn.Dropout(p=self.dropout),

            nn.Linear(1024, 512),            
            nn.BatchNorm1d(512),
            # nn.LeakyReLU(),
            nn.GELU(),
            nn.Dropout(p=self.dropout),

            nn.Linear(512, 512),            
            nn.BatchNorm1d(512),
            # nn.LeakyReLU(),
            nn.GELU(),
            nn.Dropout(p=self.dropout),

            nn.Linear(512, 256),           
            nn.BatchNorm1d(256),
            # nn.LeakyReLU(),
            nn.GELU(),
            nn.Dropout(p=self.dropout*0.5),

            nn.Linear(256, 256),            
            nn.BatchNorm1d(256),
            # nn.LeakyReLU(),
            nn.GELU(),
            nn.Dropout(p=self.dropout*0.5),

            nn.Linear(256, 128),            
            nn.BatchNorm1d(128),
            # nn.LeakyReLU(),
            nn.GELU(),
            # nn.Dropout(p=self.dropout),

            nn.Linear(128, output_size)      
        )

        if self.weight_init is not None:
            self.initialize_weights()

    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                if self.weight_init == "xavier_normal":
                    nn.init.xavier_normal_(m.weight)
                elif self.weight_init == "xavier_uniform":
                    nn.init.xavier_uniform_(m.weight)
                elif self.weight_init == "kaiming_normal":
                    nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
                elif self.weight_init == "kaiming_uniform":
                    nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
                elif self.weight_init == "uniform":
                    nn.init.uniform_(m.weight)
                else:
                    raise ValueError("Invalid weight_initialization value")

                # Initialize bias to 0
                m.bias.data.fill_(0)

    def forward(self, x):
        # Flatten to a 1D vector for each data point
        x = torch.flatten(x, start_dim=1)  # Keeps batch size, flattens the rest

        return self.model(x)

In [ ]:
# Define the input size
INPUT_SIZE  = (2*config['context'] + 1) * 28 # Why is this the case?

# Instantiate model and load to GPU
model       = Network(INPUT_SIZE, len(train_data.phonemes), config).to(device).cuda()

# Remember, you are limited to 20 million parameters for HW1 (including ensembles)
# Check to stay below 20 MIL Parameter limit
assert sum(p.numel() for p in model.parameters() if p.requires_grad) < 20_000_000, "Exceeds 20 MIL params. Any submission made to Kaggle with this model will be flagged as an AIV."

# .pth checkpoint file path can also be obtained from a locally saved .pth file. Or, you can use the checkpoint_dict obtained from the prior wandb artifact download :)
checkpoint_path = "<PATH TO YOUR CHAECKPOINT>.pth"
checkpoint_dict = torch.load(checkpoint_path)

# Loading model weights
# if isinstance(model, nn.DataParallel) model.module.load_state_dict(checkpoint_dict['model_state_dict'])
model.load_state_dict(checkpoint_dict['model_state_dict'])
# from torchsummaryX import summary

# summary(model, frames.to(device))


In [ ]:
criterion = torch.nn.CrossEntropyLoss() # loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate'],weight_decay=config['weight_decay'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode='max',factor=0.4, patience=5,threshold=0.0005) 

scaler = torch.amp.GradScaler('cuda', enabled=True)

# Loading optimizer state
optimizer.load_state_dict(checkpoint_dict['optimizer_state_dict'])
# Loading the scheduler state
scheduler.load_state_dict(checkpoint_dict['scheduler_state_dict'])

In [ ]:
# Inspect model architecture and check to verify number of parameters of your network
# try:
#     # Install and import torchsummaryX
#     !pip install torchsummaryX==1.1.0
#     from torchsummaryX import summary

#     summary(model, frames.to(device))

# except:
#     !pip install torchsummary
#     from torchsummary import summary

#     summary(model, frames[0].to(device).shape)

In [ ]:
# CLEAR RAM!!
torch.cuda.empty_cache()
gc.collect()

In [ ]:

def train(model, dataloader, optimizer, criterion, device, scaler):
    model.train()
    tloss, tacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(dataloader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Initialize Gradients
        optimizer.zero_grad()

        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        with torch.autocast(device_type=device.type, dtype=torch.float16): # bfloat16 for mac, change back to float16 for other devices
            ### Forward Propagation
            logits  = model(frames)

            ### Loss Calculation
            loss    = criterion(logits, phonemes)

        ### Backward Propagation
        scaler.scale(loss).backward()

        # OPTIONAL: You can add gradient clipping here, if you face issues of exploding gradients

        ### Gradient Descent
        scaler.step(optimizer)
        scaler.update()

        tloss   += loss.item()
        tacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        batch_bar.set_postfix(loss="{:.04f}".format(float(tloss / (i + 1))),
                              acc="{:.04f}%".format(float(tacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()


    batch_bar.close()
    tloss   /= len(dataloader)
    tacc    /= len(dataloader)


    return tloss, tacc

In [ ]:

def eval(model, dataloader, device, criterion):

    model.eval() # set model in evaluation mode
    vloss, vacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(dataloader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Move data to device (ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        # makes sure that there are no gradients computed as we are not training the model now
        with torch.inference_mode():
            ### Forward Propagation
            logits  = model(frames)
            ### Loss Calculation
            loss    = criterion(logits, phonemes)

        vloss   += loss.item()
        vacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        # Do you think we need loss.backward() and optimizer.step() here?

        batch_bar.set_postfix(loss="{:.04f}".format(float(vloss / (i + 1))),
                                acc="{:.04f}%".format(float(vacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    vloss   /= len(dataloader)
    vacc    /= len(dataloader)

    return vloss, vacc

In [ ]:
def wandb_run(model, config, name: str, resume_old_run=False, id=None):

    wandb.login(key="<YOUR KEY>") #API Key is in your wandb account, under settings (wandb.ai/settings)

    # Create your wandb run
    RESUME_OLD_RUN =resume_old_run

    if RESUME_OLD_RUN == True:
        print("Resuming previous WanDB run...")
        run = wandb.init(
            name    = name, ### Wandb creates random run names if you skip this field, we recommend you give useful names
            id     = id, ### TODO: Insert specific run id here if you want to resume a previous run
            resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
            project = "<YOUR PROJECT NAME>", ### Project should be created in your wandb account
            config  = config ### Wandb Config for your run
        )
    else:
        print("Initializing new WanDB run...")
        run = wandb.init(
            name    = name, ### Wandb creates random run names if you skip this field, we recommend you give useful names
            reinit  = True, ### Allows reinitalizing runs when you re-run this cell
            project = "<YOUR PROJECT NAME>", ### Project should be created in your wandb account
            config  = config ### Wandb Config for your run
        )

    ### Save your model architecture as a string with str(model)
    model_arch  = str(model)
    arch_filename = f"<YOUR PROJECT NAME>_{name}.txt"  # 基于name参数生成文件名
    arch_file   = open(arch_filename, "w")
    file_write  = arch_file.write(model_arch)
    arch_file.close()

    ### log it in your wandb run with wandb.save()
    wandb.save(arch_filename)
    wandb.watch(model, log="all")
    
    return run


In [ ]:
# Iterate over number of epochs to train and evaluate your model

# run = wandb_run(model, config, run_name)
run = wandb_run(model, config, run_name, resume_old_run=True, id="<ID OF WANDB RUN YOU WANT TO RECOVER>")

torch.cuda.empty_cache()
gc.collect()

best_val_acc = 0
patience = 0

for epoch in range(config['epochs']):

    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    curr_lr                 = float(optimizer.param_groups[0]['lr'])
    train_loss, train_acc   = train(model, train_loader, optimizer, criterion, device, scaler)
    val_loss, val_acc       = eval(model, val_loader, device, criterion)

    print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss, curr_lr))
    print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100, val_loss))

    ## Log metrics at each epoch in your run
    # Optionally, you can log at each batch inside train/eval functions
    # (explore wandb documentation/wandb recitation)
    wandb.log({'train_acc': train_acc*100, 'train_loss': train_loss,
            'val_acc': val_acc*100, 'valid_loss': val_loss, 'lr': curr_lr})

    # If using a scheduler, step the learning rate here, otherwise comment this line
    # Depending on the scheduler in use, you may or may not need to pass in a metric into the step function, so read the docs well
    
    if config["scheduler"] == "ReduceLROnPlateau":
        scheduler.step(val_acc)
    else:
        scheduler.step() ## or (val_acc) for StepLR
    
    ## HIGHLY RECOMMENDED: Save model checkpoint in drive and/or wandb if accuracy is better than your current best accuracy
    ## This enables you to resume training at anytime, without having to start from scratch.
    ## Refer to Recitation 0.24 to learn how to implement this: https://www.youtube.com/watch?v=-TCH0DbUEKM&list=PLp-0K3kfddPw2D5CeA09lsx_oNy9E
    
    # Save checkpoint only when validation accuracy improves
    if val_acc > best_val_acc:
        patience = 0;
        best_val_acc = val_acc
        checkpoint_path = f"{run_name}_best.pth"
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'train_acc': train_acc,
            'val_acc': val_acc,
            'config': config
        }, checkpoint_path)
        # Save to wandb
        wandb.save(checkpoint_path)
        print(f"🎉 New best model saved! Val Acc: {val_acc:.4f} -> {checkpoint_path}")
    else:
        patience += 1;
        print(f"Val Acc: {val_acc:.4f} (Best: {best_val_acc:.4f}) - No improvement, patience={patience}")

    
run.finish()

In [ ]:

def test(model, PHONEMES, test_loader, device):
    ### What you call for model to perform inference?
    model.eval() # TODO train or eval?

    ### List to store predicted phonemes of test data
    test_predictions = []

    ### Which mode do you need to avoid gradients?
    with torch.no_grad(): #TODO

        for i, mfccs in enumerate(tqdm(test_loader)):

            mfccs   = mfccs.to(device)

            logits  = model(mfccs)

            ### Get most likely predicted phoneme with argmax
            predicted_phonemes = torch.argmax(logits, dim=1)

            ### How do you store predicted_phonemes with test_predictions? HINT: look at the eval() function from earlier
            # Remember the phonemes were converted from strings to their corresponding integer indices earlier, and the results of the argmax is a list of the integer indices of the predicted phonemes.
            # So how do you get and store the actual predicted phonemes (strings NOT integers)
            # TODO: Convert predicted_phonemes (integer indices from argmax) back to phoneme strings and append them to test_predictions
            for phoneme_index in predicted_phonemes:
                test_predictions.append(PHONEMES[phoneme_index])

            # raise NotImplementedError(
            #     "TODO: convert predicted_phonemes integer indices -> phoneme strings and append to test_predictions. "
            #     "Replace this exception with the correct code implementation."
            # )

    ## SANITY CHECK
    sample_predictions = test_predictions[:10]
    if not isinstance(sample_predictions[0], str):
        print(f"❌ ERROR: Predictions should be phoneme STRINGS, not {type(sample_predictions[0]).__name__}!")
        print(f"   You need to convert integer indices to their corresponding phoneme strings")
        print(f"   Hint: Look at the eval() function to get the idea")

    # Print a preview of predictions for manual inspection
    print("\nSample predictions:", sample_predictions)
    print("\nPredictions Generated successfully!")

    return test_predictions

In [ ]:
# Generate model test predictions

predictions = test(model, test_loader)

In [ ]:
### Create CSV file with predictions

with open("./submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, predictions[i]))

    print("submission.csv file created successfully!")